In [2]:
# Defined seperator conditions
import math
pr = 4050 #psia
tr = 255 #deg F
cgr = 13 #stb/mmscf
cond_api = 69 #deg API
salinity = 36200 #ppm TDS
psep = 1000 #psia
tsep = 80 #deg F
gamma_sep = 0.7 #From Problem 21

In [34]:
# Solution

# Need to first estimate amount and sg of gas from separator at 1,000psia and 80 deg F
A1 = ((psep/18.2+1.4)*10**(0.0125*cond_api-0.00091*tsep))**1.205 # Eq 3.61b
A2 = 0.25 + 0.02 * cond_api # Note, worked  equation has 0.2, it should actually be 0.02 per original in chapter 3
A3 = -3.57e-6 * cond_api

Rsplus =  (A1*A2)/(1-(A1*A3)) # Eq 3.63, Additional gas expected to be released from 1st stage separator oil
# **Note: Not sure what happens here, as the constituent A1, A2 & A3 match with worked answers, but the Rs+ is much different
# ** Checking consistency with equations 3.61a and 3.62, it appears that the worked solution is incorrect
# ** Eq 3.61a: A1 * SG+ = 1,152 * 0.985 = 1,134 which does not equal 2,620 scf/stb

gamma_g_plus = A2 + A3 * Rsplus #Eq 3.62, Gas gravity of the expected solution gas to be released from the condensate

Rs1 = 1/(cgr*1e-6) # scf/stb from separator
Rp = 1/(Rs1 + Rsplus) # Eq 3.60, Total producing OGR (including gas from 1st stage sep + gas expected from 1st stage condensate)
sg_avg = (gamma_sep*Rs1 + gamma_g_plus*Rsplus)/(Rs1 + Rsplus) # Eq 3.64, Avg separator gas gravity 

print('A1:', round(A1,1), ', A2:', round(A2,2), ', A3:', A3, ', Rs_plus:',round(Rsplus,1))
print('Avg surface sep gas gravity:', round(sg_avg,3), 'sg, rel to air = 1.0')

A1: 1151.8 , A2: 1.63 , A3: -0.00024633 , Rs_plus: 1462.5
Avg surface sep gas gravity: 0.711 sg, rel to air = 1.0


In [56]:
# Brine dissolved gas content
Rsw0 = 17.5 #scf gas / stb fresh water, Assumed by interpolating Figures 9.1 / 9.2 with Sg = 0.711
ks = 0.1813 - 7.692e-4*tr + 2.6614e-6*tr**2 - 2.612e-9*tr**3 # Salting out coefficient, Eq 9.10 for methane/NaCl
Rs = Rsw0 * 10**(-17.1e-6*ks*salinity) # scf gas / stb brine Eq 9.9
print('Dissolved gas in brine:', round(Rs,1), 'scf gas / stb brine')

Dissolved gas in brine: 14.9 scf gas / stb brine


In [57]:
# Brine density
tsc = 60 #deg F
tsc_K = (tsc-32)/1.8 + 273 # Standard temp in degrees Kelvin - required for Eq 9.14

A0 = 5.91635 - 0.01035794*tsc_K + 0.9270048e-5*tsc_K**2 - 1127.522/tsc_K + 100674.1/(tsc_K**2)
A1 = -2.5166 + 0.0111766*tsc_K - 0.170552e-4*tsc_K**2
A2 = 2.84851 - 0.0154305*tsc_K + 0.223982e-4*tsc_K**2
ws = salinity * 1e-6
vw = A0 + A1 * ws + A2 * ws**2 # cm3/g, Eq 9.14
rhow = 1/vw # g/cm3 **Note, the example has a misprint insofar that it has identical vw and rhow, which only makes sense when = 1

tr_K = (tr-32)/1.8 + 273
A0 = 5.91635 - 0.01035794*tr_K + 0.9270048e-5*tr_K**2 - 1127.522/tr_K + 100674.1/(tr_K**2)
A1 = -2.5166 + 0.0111766*tr_K - 0.170552e-4*tr_K**2
A2 = 2.84851 - 0.0154305*tr_K + 0.223982e-4*tr_K**2
vw0 = A0 + A1 * ws + A2 * ws**2 # Eq 9.14
rhow0 = 1/vw0

print('Brine density at standard conditions:', round(rhow,5), 'g/cm3')
print('Brine density at reservoir temperature:', round(rhow0,5), 'g/cm3')

Brine density at standard conditions: 1.02504 g/cm3
Brine density at reservoir temperature: 0.96488 g/cm3


In [71]:
# Compressibility - start with gas-free Cw*
A0 =1e6*(0.314 + 0.58*ws + 1.9e-4*tr - 1.45e-6*tr**2)
A1 = 8 + 50*ws - 0.125*ws*tr 
cw_star = 1/(A0 + A1 * pr)

#Bw without dissolved gas
Bw0 = rhow / rhow0 # FVF of brine at reservoir temperature, but atmospheric pressure
Bw_star = Bw0 * (1 + (A1/A0)*pr)**(-1/A1) # Eq 9.18, FVF of brine at reservoir temp AND reservoir pressure
# ** Note: Example has a minus in the exponent, which is not present in the original equation
# ** Checking independat correlation, I believe that the example representation is correct, and the equation requires the minus

# Note below correlations are against pure water, hence we use Rsw0
Bw = Bw_star * (1 + 0.0001 * Rsw0**1.5) #Eq 9.19
Cw = cw_star * (1 + 0.00877*Rsw0) # Eq 9.20

print('Brine FVF:', round(Bw,4), 'rb/stb')
print('Brine Compressibility:', round(Cw*1e6,2), 'e-06 psi-1')

# Note: Comments in Chapter 9 suggest that Eq 9.19 and 9.20 are only valid for undersaturated oil/water systems, which raises
# suspicion that not entirely appropriate to use here, suggesting instead application of Eq 9.21
# I would further point out that using the correlations suggested by McCain, Spivey & Len in 'Petreoleum reservoir Fluid Properties'
# I get higher compressibilities for this saturated brine, being 4.96 x 10^-6 psi-1



Brine FVF: 1.0561 rb/stb
Brine Compressibility: 3.56 e-06 psi-1
